# Rozliczanie podatków od kryptowalut w Polsce

## Konfiguracja

Konfigurację zacznij od wprowadzenia swoich kluczy API do giełd w pliku .env.
Plik konfiguracyjny stworzysz na bazie pliku .env.example.
Podaj też ścieżkę pliku csv, który zawiera listę transferów kryptowalutowych (obecnie obsługujemy format eksportu z portfela coinomi).
Program zakłada że wszystkie transfery wychodzące w pliku csv to zakupy.
W przyszłości dodana zostana funkcja oznaczania czy dany transfer był zakupem.
Należy także dodać funkcjonalność importu trasakcji z kary ledger cl card.

In [12]:
ROK = 2024
FIAT_CURRENCY_SYMBOLS = {'PLN', 'USD', 'EUR', 'CHF', 'GBP'}
nierozliczone_koszty_z_roku_2023 = 58857.12
# W portfelu coinomi są transfery zakupowe.
coinomi_transfers_csv_path = f"~/kryptorozliczator/{ROK}/input/transakcje_coinomi_do_27.04.2025.csv"
exchanges = ["zonda", "bitfinex"]

## Importy

In [13]:
from kryptorozliczator.exchange_interfaces.exchange_interface import ExchangeInterface
from kryptorozliczator.wallet_interfaces.transfers import TransfersInterface
from kryptorozliczator.rates.crypto_rates import get_crypto_exchange_rate
from kryptorozliczator.rates.nbp_rates import get_nbp_exchange_rate

import pandas as pd
import requests
from datetime import datetime, timedelta
import math
from pathlib import Path

interfaces = [ExchangeInterface(e) for e in exchanges]
wallet_interface = TransfersInterface()
wallets = ["coinomi_spending"]
output_dir = Path.home() / "kryptorozliczator" / str(ROK) / "output"
intermediate_output_dir = output_dir / "intermediate"
output_dir.mkdir(parents=True, exist_ok=True)
intermediate_output_dir.mkdir(parents=True, exist_ok=True)

## Przydatne funkcje

In [14]:
def filter_pln_pair(symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part == 'PLN':
            return True
    return False

def filter_fiat_pair(symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part in FIAT_CURRENCY_SYMBOLS:
            return True
    return False

def filter_currency_pair(symbol, currency_symbol):
    for symbol_part in symbol.split('/'):
        if symbol_part == currency_symbol:
            return True
    return False


## Pobranie historii transferów z portfeli

In [ ]:
for wallet in wallets:
    # print(f"Pobieranie transakcji z portfela {wallet}")
    # transfers = wallet_interface.get_wallet_transfers(wallet, ROK)
    # print(transfers)
    print("This feateure is still WIP")
    # transfers_df = pd.DataFrame(transfers)
    # transfers_df["wallet"] = interface.wallet_name
    # all_transfers_df = pd.concat([all_transfers_df, transfers_df])



## Pobranie historii transferów z CSV

In [ ]:
all_transfers_df = pd.DataFrame()

# Import transactions from CSV
coinomi_transfers_df = pd.read_csv(coinomi_transfers_csv_path)
coinomi_transfers_df["wallet"] = "coinomi_spending"
all_transfers_df = pd.concat([all_transfers_df, coinomi_transfers_df])

# Filter by date
all_transfers_df = all_transfers_df[(all_transfers_df['Time(ISO8601-UTC)'] >= f"{ROK}-01-01") & 
                                  (all_transfers_df['Time(ISO8601-UTC)'] < f"{ROK+1}-01-01")]
# Filter by negative value
outgoing_transfers_df = all_transfers_df[all_transfers_df['Value'] < 0]

display(outgoing_transfers_df)

## Obliczenie podatku z wydanych kryptowalut

In [ ]:
# Add column with PLN value of cryptocurrency PLN on the transaction day
outgoing_transfers_df['Rate [PLN]'] = 0.0
outgoing_transfers_df['PLN Value'] = 0.0
for idx, row in outgoing_transfers_df.iterrows():
    date = pd.to_datetime(row['Time(ISO8601-UTC)']).date()
    asset = row['Symbol']
    price = get_crypto_exchange_rate(asset, 'PLN', str(date))
    # Convert Value to float if it's not already
    value = float(math.fabs(float(row['Value'])))
    # Update the dataframe directly using loc to avoid the sequence multiplication error
    outgoing_transfers_df.loc[idx, 'Rate [PLN]'] = price
    value_pln = value * price
    outgoing_transfers_df.loc[idx, 'PLN Value'] = value_pln
    
display(outgoing_transfers_df)

# Save outgoing_transfers_df to csv
outgoing_transfers_df.to_csv(intermediate_output_dir / 'outgoing_transfers.csv', index=False)


## Pobranie historii konwersji walut z giełd

In [ ]:
all_transactions_df = pd.DataFrame()

for interface in interfaces:
    print(f"Pobieranie transakcji z {interface.exchange_name}")
    transactions = interface.get_transaction_history(ROK)
    transactions_df = pd.DataFrame(transactions)
    transactions_df["exchange"] = interface.exchange_name
    all_transactions_df = pd.concat([all_transactions_df, transactions_df])


## Wyświetlenie historii konwersji walut z giełd

In [ ]:
## Display transactions as a table
display(all_transactions_df)

## Display transaction in print
# print(all_transactions_df)


## Zapis tabel wynikowych CSV

In [ ]:
import os

# Create a CSV file for the transactions
csv_file_path = intermediate_output_dir / 'conversions.csv'
all_transactions_df.to_csv(csv_file_path, index=False)


## Obliczenie przychodów i kosztów

In [ ]:
def calculate_transaction_value_and_fee(row: pd.Series):
    currency_symbol, base_currency_symbol = row['symbol'].split('/')
    transaction_date = datetime.fromtimestamp(row['timestamp']/1000)

    # mean NBP rate should be taken from 1 day before transaction date
    nbp_rate_date = transaction_date - timedelta(days=1)

    transaction_value_fiat = row['cost']
    if base_currency_symbol == 'PLN':
        transaction_value_pln = transaction_value_fiat
    elif base_currency_symbol in FIAT_CURRENCY_SYMBOLS:
        exchange_rate = get_nbp_exchange_rate(base_currency_symbol, nbp_rate_date)
        transaction_value_pln = transaction_value_fiat * exchange_rate
    else:
        raise ValueError(f"Unsupported currency symbol: {row['symbol']}")

    fee = row['fee']
    fee_currency = fee['currency']
    fee_cost = float(fee['cost'])
    if fee_currency == 'PLN':
        fee_value_pln = fee_cost
        fee_value_fiat = fee_cost
    elif fee_currency in FIAT_CURRENCY_SYMBOLS:
        exchange_rate = get_nbp_exchange_rate(fee_currency, nbp_rate_date)
        fee_value_pln = fee_cost * exchange_rate
        fee_value_fiat = fee_cost
    else: # fee in crypto, convert to fiat, then to PLN
        crypto_to_fiat_exchange_rate = row['price']
        fee_value_fiat = fee_cost * crypto_to_fiat_exchange_rate
        if base_currency_symbol == 'PLN':
            fee_value_pln = fee_value_fiat
        elif base_currency_symbol in FIAT_CURRENCY_SYMBOLS:
            exchange_rate = get_nbp_exchange_rate(base_currency_symbol, nbp_rate_date)
            fee_value_pln = fee_value_fiat * exchange_rate
        else:
            raise ValueError(f"Unsupported currency symbol: {row['symbol']}")
    return {'transaction_value_fiat': transaction_value_fiat,
            'transaction_value_pln': transaction_value_pln,
            'fee_value_fiat': fee_value_fiat,
            'fee_value_pln': fee_value_pln}

def calculate_transaction_totals_for_single_currency(df, currency_symbol):
    # Choose only transactions in the pair to fiat currency, the rest don't affect tax
    df = df[df['symbol'].apply(lambda x: filter_currency_pair(x, currency_symbol))]

    # Grupowanie transakcji na kupno i sprzedaż
    buys = df[df['side'] == 'buy']
    sells = df[df['side'] == 'sell']

    total_buy_cost_pln = 0
    total_buy_cost_original_currency = 0
    total_sell_revenue_pln = 0
    total_sell_revenue_original_currency = 0
    total_fee_fiat = 0
    total_fee_pln = 0
    for _, row in buys.iterrows():
        transaction_value_and_fee = calculate_transaction_value_and_fee(row)
        total_buy_cost_pln += transaction_value_and_fee['transaction_value_pln']
        total_buy_cost_original_currency += transaction_value_and_fee['transaction_value_fiat']
        total_fee_fiat += transaction_value_and_fee['fee_value_fiat']
        total_fee_pln += transaction_value_and_fee['fee_value_pln']

    for _, row in sells.iterrows():
        transaction_value_and_fee = calculate_transaction_value_and_fee(row)
        total_sell_revenue_pln += transaction_value_and_fee['transaction_value_pln']
        total_sell_revenue_original_currency += transaction_value_and_fee['transaction_value_fiat']
        total_fee_fiat += transaction_value_and_fee['fee_value_fiat']
        total_fee_pln += transaction_value_and_fee['fee_value_pln']
    
    return {'total_buy_cost_pln': total_buy_cost_pln,
            'total_buy_cost_original_currency': total_buy_cost_original_currency,
            'total_sell_revenue_pln': total_sell_revenue_pln,
            'total_sell_revenue_original_currency': total_sell_revenue_original_currency,
            'total_fee_fiat': total_fee_fiat,
            'total_fee_pln': total_fee_pln}

total_buy_cost_pln = 0
total_sell_revenue_pln = 0
total_fee_pln = 0
for currency_symbol in FIAT_CURRENCY_SYMBOLS:
    totals = calculate_transaction_totals_for_single_currency(all_transactions_df, currency_symbol)
    if totals['total_buy_cost_pln'] == 0 and totals['total_sell_revenue_pln'] == 0 and totals['total_fee_pln'] == 0:
        continue
    currency_summary_df = pd.DataFrame({
        'Kategoria': [f'Koszt zakupu (PLN)', f'Przychód ze sprzedaży (PLN)', f'Opłaty (PLN)', f'Koszt w {currency_symbol}', f'Przychód w {currency_symbol}'],
        'Wartość (PLN)': [
            totals['total_buy_cost_pln'],
            totals['total_sell_revenue_pln'],
            totals['total_fee_pln'],
            totals['total_buy_cost_original_currency'],
            totals['total_sell_revenue_original_currency']
        ]
    })
    print(f"\nPodsumowanie wymian na giełdzie dla waluty {currency_symbol}:")
    display(currency_summary_df)
    currency_summary_df.to_csv(intermediate_output_dir / f'conversions_summary_{currency_symbol}.csv', index=False)

    total_buy_cost_pln += totals['total_buy_cost_pln']
    total_sell_revenue_pln += totals['total_sell_revenue_pln']
    total_fee_pln += totals['total_fee_pln']

print(f"Podsumowanie wymian na giełdzie dla wszystkich walut:")
all_conversions_summary_df = pd.DataFrame({
    'Kategoria': ['Koszt zakupu (PLN)', 'Przychód ze sprzedaży (PLN)', 'Opłaty (PLN)'],
    'Wartość (PLN)': [total_buy_cost_pln, total_sell_revenue_pln, total_fee_pln]
})
display(all_conversions_summary_df)
all_conversions_summary_df.to_csv(intermediate_output_dir / 'conversions_summary.csv', index=False)

print(f"\nPodsumowanie zakupów dóbr i usług przy użyciu kryptowalut:")
goods_and_services_buy_summary_df = pd.DataFrame({
    'Kategoria': ['Przychód ze sprzedaży (PLN)'],
    'Wartość (PLN)': [
        outgoing_transfers_df['PLN Value'].sum(),
    ]
})
display(goods_and_services_buy_summary_df)
goods_and_services_buy_summary_df.to_csv(intermediate_output_dir / 'goods_and_services_buy_summary.csv', index=False)
total_sell_revenue_pln += outgoing_transfers_df['PLN Value'].sum()

summary_df = pd.DataFrame({
    'Category': ['Koszt zakupu', 'Przychód ze sprzedaży', 'Opłaty', 'Koszt całkowity', 'Zysk całkowity'],
    'Value (PLN)': [
        total_buy_cost_pln,
        total_sell_revenue_pln, 
        total_fee_pln,
        total_buy_cost_pln + total_fee_pln,
        total_sell_revenue_pln - total_buy_cost_pln - total_fee_pln
    ]
})
print("\nPodsumowanie:")
display(summary_df)
summary_df.to_csv(output_dir / 'TOTALS.csv', index=False)

## Co wpisać do PIT38

In [ ]:

PIT38 = pd.DataFrame({
    'Kategoria': ['Przychód z kryptowalut w PLN (34)',
                 'Koszty uzyskania przychodu poniesione w roku podatkowym (35)',
                 'Koszty uzyskania przychodu poniesione w latach ubiegłych i niepotrącone w poprzednim roku podatkowym (36)',
                 'Dochód (37)',
                 'Koszty uzyskania przychodu, które nie zostały potrącone w roku podatkowym (38)'],
    'Value (PLN)': [
        total_sell_revenue_pln,
        total_buy_cost_pln + total_fee_pln,
        nierozliczone_koszty_z_roku_2023,
        total_sell_revenue_pln - total_buy_cost_pln - total_fee_pln - nierozliczone_koszty_z_roku_2023,
        max(0, total_buy_cost_pln + total_fee_pln + nierozliczone_koszty_z_roku_2023 - total_sell_revenue_pln)
    ]
})
pd.set_option('display.max_colwidth', None)
display(PIT38)
PIT38.to_csv(output_dir / 'PIT38.csv', index=False)